# Law of Large Numbers

> According to the law, the average of the results obtained from a large number of trials should be close to the expected value and will tend to become closer to the expected value as more trials are performed.
> 
> -- <cite>Wikipedia</cite>

The *Law of Large Numbers* allows statistical inference to be possible. It guarantees that, under certain conditions, the moments we calculate from samples will converge asymptotically to the true moments. Of course, we ought to ask ourselves the following questions:

1. Under which conditions the Law of Large Number works?
2. When the Law of Large Numbers works, how *fast* the sample moment converges to the true one?

## Convergence of Moments of the Normal Distribuition: An experimental approach

Let's develop a feeling of how moments converge using the Normal distribuition as an example:

In [45]:
from sympy.stats import Normal
from sympy.stats import sample
import plotly.graph_objects as go

NUMBER_OF_SAMPLES = 1000
TRUE_MEAN = 5
TRUE_STANDARD_DEVIATION = 3 # Remember: variance = standard_deviation^2

random_variable = Normal('X', TRUE_MEAN, TRUE_STANDARD_DEVIATION)

x_axis = list(range(0,NUMBER_OF_SAMPLES))
experiments = [float(sample(random_variable)) for x in x_axis]

# Plot
fig = go.Figure(data=go.Scatter(x=x_axis, y=experiments))
fig.show()

`experiments` contains samples of a Normally Distribuited random variable with `TRUE_MEAN` mean and variance equals `TRUE_STANDARD_DEVIATION`^2. To calculate the first moment, we can use the recursive mean formula:

In [2]:
from sympy import symbols

past_mean, index, current_sample = symbols("M_{n-1} n Xn")

recursive_mean  = past_mean * (index-1) / index + current_sample / index
recursive_mean

M_{n-1}*(n - 1)/n + Xn/n

Note that what we are going to do isn't computationally efficient. That said, we want to see how Mn evolves as we increase the number of samples:

In [46]:
means = [0]

for index in range(1,NUMBER_OF_SAMPLES):
    current_mean = (means[index-1] *(index-1)/index) + (experiments[index]/index)
    means.append(current_mean) 

# Plot
fig = go.Figure(data=go.Scatter(x=x_axis, y=means))
fig.show()

We can also plot the relative error (%) evolution:


In [47]:
errors = [100*(mean-TRUE_MEAN)/TRUE_MEAN for mean in means]

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_axis, y=errors,
                    mode='lines',
                    name='Error relative to the true mean (%)'))
fig.show()

We can also see the evolution of each term of the recusive mean formula:

In [48]:
means = [0]
left_terms = [0]
right_terms = [0]

for index in range(1,NUMBER_OF_SAMPLES):
    current_left_term = means[index-1] *(index-1)/index 
    current_right_term = experiments[index]/index
    current_mean = current_left_term + current_right_term
    left_terms.append(current_left_term)
    right_terms.append(current_right_term)
    means.append(current_mean) 

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_axis, y=left_terms,
                    mode='lines',
                    name='Left Term'))
fig.add_trace(go.Scatter(x=x_axis, y=right_terms,
                    mode='lines',
                    name='Right Term'))
fig.show()

From the recursive formula we can see that as `n` gets bigger `(n-1)/n` approaches `1` and `Xn/n` approaches zero. Hence, the formula becomes `Mn = Mn-1` and `Mn` converges.

An important result is that the convergence of the first moment, for any distribuition, depends only on the asymptotic behaviour of the term `Xn/n`. If `Xn` grows faster or with the same speed than `n`, then the series will diverge. Otherwise, it will converge. But since `X` is a random variable, we must ask ourselves the following question: 

> What is the probability of `Xn` being larger than `n`, as n gets larger?  

Note that this is exactly the question a random variable's PDF answers. For the Normal Distribuition case, the probability of `Xn` being larger than `n` decays exponentially to zero. Hence:

1. The sample mean converges to the true mean.
2. The sample mean converges to the true mean *very fast*.

Naturally this isn't a proof of the Law of Large Numbers. It says nothing about other moments nor other distribuitions. But it gives us a good intuition of *how* and *why* LLN works.


Now let's do the same for the second order moment, also known as variance. We can calculate the variance recursively using the following formula, where:

- past_mean M_{n-1} is the mean of the previous n−1 samples.
- current_mean M_{n} is the mean after adding the current sample.
- past_m2 M2_{n-1} is the second-order moment of the previous samples.
- index (n) represents the total number of samples added so far
- current_sample Xn is the new sample being added

In [12]:
from sympy import simplify


# Define symbols
past_mean, current_mean, past_m2, index, current_sample = symbols("M_{n-1} M_{n} M2_{n-1} n Xn")

# Recursive formula for the second-order moment M2
delta = current_sample - past_mean
recursive_m2 = past_m2 + delta * (current_sample - current_mean)

# Recursive formula for the variance
variance = recursive_m2 / index

# Display the recursive formulas
simplify(variance)

(M2_{n-1} + (M_{n-1} - Xn)*(M_{n} - Xn))/n

In [49]:
# Variables for recursive computation
means = [0]  # List to store the mean
m2s = [0]    # List to store the second-order moment (M2)
variances = [0]  # List to store the variance

for index in range(1, NUMBER_OF_SAMPLES):
    # Previous mean
    past_mean = means[index - 1]
    
    # Current sample
    current_sample = experiments[index]
    
    # Update mean recursively
    current_mean = past_mean + (current_sample - past_mean) / index
    means.append(current_mean)
    
    # Update M2 recursively
    past_m2 = m2s[index - 1]
    delta = current_sample - past_mean
    m2 = past_m2 + delta * (current_sample - current_mean)
    m2s.append(m2)
    
    # Compute variance
    current_variance = m2 / index if index > 1 else 0
    variances.append(current_variance)

# Plot mean and variance
x_axis = list(range(NUMBER_OF_SAMPLES))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_axis, y=means, mode='lines', name='Mean'))
fig.add_trace(go.Scatter(x=x_axis, y=variances, mode='lines', name='Variance'))
fig.update_layout(title='Recursive Mean and Variance',
                  template='plotly_dark',
                  xaxis_title='Number of Samples',
                  yaxis_title='Value')
fig.show()

Just like in the first order moment, the variance converges to its true value as we add more samples to our estimation. Note, however, that the convergence looks "noiser" and takes longer. In fact, even after 1000 samples our error still higher than 4%. That's the first catch on the Law of Large Numbers: higher order moments require more samples to archieve the same level of accuracy as the first order moment. That follows from the recursive formula: the variance is a function of squared deviantions from the mean, which introduces a squared term that grows faster and takes longer to average out across samples. Let's do the same experiment for the first nth moments of our distribution:

In [50]:
import math
import numpy as np


TRUE_VARIANCE = TRUE_STANDARD_DEVIATION ** 2

# Parameter to specify the number of moments to compute
NUMBER_OF_MOMENTS = 5  # You can change this to compute more or fewer moments

# Initialize lists to store the moments
moments = [[] for _ in range(NUMBER_OF_MOMENTS)]

# Compute the moments recursively
for n in range(1, NUMBER_OF_SAMPLES + 1):
    sample_subset = np.array(experiments[:n])  # Convert the sample subset to a NumPy array
    for k in range(1, NUMBER_OF_MOMENTS + 1):  # Compute moments from 1 to NUMBER_OF_MOMENTS
        moment_k = np.mean(sample_subset**k)  # k-th raw moment around 0
        moments[k-1].append(moment_k)

# Calculate the true raw moments for a normal distribution using the exact formula
true_moments = []
for k in range(1, NUMBER_OF_MOMENTS + 1):
    true_moment = sum(
        (math.factorial(k) / (math.factorial(k - 2 * m) * math.factorial(m) * (2**m))) *
        (TRUE_MEAN**(k - 2 * m)) * (TRUE_VARIANCE**m)
        for m in range(0, (k // 2) + 1)
    )
    true_moments.append(true_moment)

# Prepare the plot using Plotly
fig = go.Figure()

# Add each moment to the plot
for k in range(1, NUMBER_OF_MOMENTS + 1):
    fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                             y=moments[k-1], 
                             mode='lines', 
                             name=f'{k}-th Moment (Estimated)'))

# Plot the true moments as horizontal lines
for k in range(1, NUMBER_OF_MOMENTS + 1):
    fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                             y=[true_moments[k-1]] * NUMBER_OF_SAMPLES, 
                             mode='lines', 
                             name=f'{k}-th Moment (True)', 
                             line=dict(dash='dash')))

# Add layout details
fig.update_layout(
    title=f'Convergence of the First {NUMBER_OF_MOMENTS} Moments (with True Values)',
    xaxis_title='Number of Samples',
    yaxis_title='Moment Value',
    legend_title='Moments',
    template='plotly_dark',
    hovermode='x unified'
)

# Show the plot
fig.show()

Again, we can see higher order moments converge much slower, as expected due higher order terms in their formulas. Finally, convergence also works differently for distributions that are not gaussian. If a distribution's PDF converges to zero more slowly, then moment convergence will suffer as well.

In [53]:
LAMBDA_EXPONENTIAL = 1/5
LAMBDA_POISSON = 5
DEGREES_OF_FREEDOM_T = 2  # Degrees of freedom for Student's t-distribution
PARETO_SHAPE = 2  # Shape parameter for Pareto distribution (must be > 1 for mean to exist)

# Generate samples from different distributions
samples_normal = np.random.normal(loc=TRUE_MEAN, scale=TRUE_STANDARD_DEVIATION, size=NUMBER_OF_SAMPLES)
samples_uniform = np.random.uniform(low=0, high=10, size=NUMBER_OF_SAMPLES)  # Mean is already 5
samples_exponential = np.random.exponential(scale=1/LAMBDA_EXPONENTIAL, size=NUMBER_OF_SAMPLES)  # Mean is 5
samples_laplace = np.random.laplace(loc=TRUE_MEAN, scale=TRUE_STANDARD_DEVIATION, size=NUMBER_OF_SAMPLES)
samples_poisson = np.random.poisson(lam=LAMBDA_POISSON, size=NUMBER_OF_SAMPLES)

# Adjust the distributions with means that don't match 5
samples_student_t = np.random.standard_t(df=DEGREES_OF_FREEDOM_T, size=NUMBER_OF_SAMPLES) + TRUE_MEAN  # Center around 5
samples_pareto = (np.random.pareto(a=PARETO_SHAPE, size=NUMBER_OF_SAMPLES) + 1) * (TRUE_MEAN / (PARETO_SHAPE / (PARETO_SHAPE - 1)))  # Scale to mean 5

# Fat-tailed distribution (Cauchy remains as is, mean is undefined)
samples_cauchy = np.random.standard_cauchy(size=NUMBER_OF_SAMPLES) + TRUE_MEAN  # Center around 5, but mean is still undefined

# Initialize lists to store the sample means for each distribution
means_normal = []
means_uniform = []
means_exponential = []
means_laplace = []
means_poisson = []
means_cauchy = []
means_student_t = []
means_pareto = []

# Compute the sample means recursively for each distribution
for n in range(1, NUMBER_OF_SAMPLES + 1):
    means_normal.append(np.mean(samples_normal[:n]))
    means_uniform.append(np.mean(samples_uniform[:n]))
    means_exponential.append(np.mean(samples_exponential[:n]))
    means_laplace.append(np.mean(samples_laplace[:n]))
    means_poisson.append(np.mean(samples_poisson[:n]))
    means_cauchy.append(np.mean(samples_cauchy[:n]))
    means_student_t.append(np.mean(samples_student_t[:n]))
    means_pareto.append(np.mean(samples_pareto[:n]))

# Prepare the plot using Plotly
fig = go.Figure()

# Plot the convergence of the sample means for each distribution
fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_normal, 
                         mode='lines', 
                         name='Normal Distribution'))

fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_uniform, 
                         mode='lines', 
                         name='Uniform Distribution'))

fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_exponential, 
                         mode='lines', 
                         name='Exponential Distribution'))

fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_laplace, 
                         mode='lines', 
                         name='Laplace Distribution'))

fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_poisson, 
                         mode='lines', 
                         name='Poisson Distribution'))

fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_cauchy, 
                         mode='lines', 
                         name='Cauchy Distribution'))

fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_student_t, 
                         mode='lines', 
                         name="Student's t-Distribution (df=2)"))

fig.add_trace(go.Scatter(x=list(range(1, NUMBER_OF_SAMPLES + 1)), 
                         y=means_pareto, 
                         mode='lines', 
                         name='Pareto Distribution (shape=2)'))

# Add layout details
fig.update_layout(
    title='Convergence of Sample Means for Different Distributions (Including Fat-Tailed Distributions)',
    xaxis_title='Number of Samples',
    yaxis_title='Sample Mean',
    legend_title='Distributions',
    template='plotly_dark',
    hovermode='x unified'
)

# Show the plot
fig.show()



All ditributions have the same mean, with one exception: Cauchy Ditributions do not have defined moments. Hence, they never converge to anything. Interestingly, looking at its estimation curve, we could be tricked into believing in convergence for finite periods of time.

Another interesting behavior: some fat tailed distributions, such as Pareto, seem to be almost converge to its true value, just to diverge again by large amounts at certain times. That's because those distributions can produce very large values with non-negligible probability, leading to a very slow convergence of the sample mean.

If you didn't see anything weird in the Pareto convergence, that's fine. It could be the case that no extreme values were generated in the sample. To avoid that, in the next example we run a a Monte Carlo simulation for all distributions, to make sure we get enough samples. We also plot everything in a logarithm scale so the differences in convergence speed are more aparent.

In [56]:
import numpy as np
import plotly.graph_objects as go

# Parameters
NUMBER_OF_SAMPLES = 100000  # Number of samples per run
NUM_RUNS = 100  # Number of Monte Carlo runs to observe convergence variability
TRUE_MEAN = 5
TRUE_STANDARD_DEVIATION = 3
LAMBDA_EXPONENTIAL = 1/5
LAMBDA_POISSON = 5
DEGREES_OF_FREEDOM_T = 2
PARETO_SHAPE = 2

# Function to compute the absolute deviation from the true mean over multiple runs for a given distribution
def compute_convergence_distributions(num_runs, num_samples, distribution, **kwargs):
    dev = np.zeros((num_runs, num_samples))
    for run in range(num_runs):
        if distribution == 'normal':
            samples = np.random.normal(loc=TRUE_MEAN, scale=TRUE_STANDARD_DEVIATION, size=num_samples)
        elif distribution == 'uniform':
            samples = np.random.uniform(low=0, high=10, size=num_samples)
        elif distribution == 'exponential':
            samples = np.random.exponential(scale=1/LAMBDA_EXPONENTIAL, size=num_samples)
        elif distribution == 'laplace':
            samples = np.random.laplace(loc=TRUE_MEAN, scale=TRUE_STANDARD_DEVIATION, size=num_samples)
        elif distribution == 'poisson':
            samples = np.random.poisson(lam=LAMBDA_POISSON, size=num_samples)
        elif distribution == 'student_t':
            samples = np.random.standard_t(df=DEGREES_OF_FREEDOM_T, size=num_samples) + TRUE_MEAN
        elif distribution == 'pareto':
            samples = (np.random.pareto(a=PARETO_SHAPE, size=num_samples) + 1) * (TRUE_MEAN / (PARETO_SHAPE / (PARETO_SHAPE - 1)))
        elif distribution == 'cauchy':
            samples = np.random.standard_cauchy(size=num_samples) + TRUE_MEAN
        else:
            raise ValueError("Unknown distribution type.")

        for n in range(1, num_samples + 1):
            dev[run, n - 1] = abs(np.mean(samples[:n]) - TRUE_MEAN)
    return dev

# Compute deviations for each distribution across multiple runs
dev_normal = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'normal')
dev_uniform = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'uniform')
dev_exponential = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'exponential')
dev_laplace = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'laplace')
dev_poisson = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'poisson')
dev_cauchy = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'cauchy')
dev_student_t = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'student_t')
dev_pareto = compute_convergence_distributions(NUM_RUNS, NUMBER_OF_SAMPLES, 'pareto')

# Prepare the plot using Plotly
fig = go.Figure()

# Plot the log of absolute deviations for each distribution across multiple runs
def add_distribution_trace(fig, dev, name, num_runs, num_samples):
    for run in range(num_runs):
        fig.add_trace(go.Scatter(x=list(range(1, num_samples + 1)), 
                                 y=np.log10(dev[run, :]), 
                                 mode='lines', 
                                 opacity=0.2, 
                                 showlegend=(run == 0), 
                                 name=name))

add_distribution_trace(fig, dev_normal, 'Normal Distribution', NUM_RUNS, NUMBER_OF_SAMPLES)
add_distribution_trace(fig, dev_uniform, 'Uniform Distribution', NUM_RUNS, NUMBER_OF_SAMPLES)
add_distribution_trace(fig, dev_exponential, 'Exponential Distribution', NUM_RUNS, NUMBER_OF_SAMPLES)
add_distribution_trace(fig, dev_laplace, 'Laplace Distribution', NUM_RUNS, NUMBER_OF_SAMPLES)
add_distribution_trace(fig, dev_poisson, 'Poisson Distribution', NUM_RUNS, NUMBER_OF_SAMPLES)
add_distribution_trace(fig, dev_cauchy, 'Cauchy Distribution', NUM_RUNS, NUMBER_OF_SAMPLES)
add_distribution_trace(fig, dev_student_t, "Student's t-Distribution (df=2)", NUM_RUNS, NUMBER_OF_SAMPLES)
add_distribution_trace(fig, dev_pareto, 'Pareto Distribution (shape=2)', NUM_RUNS, NUMBER_OF_SAMPLES)

# Add layout details
fig.update_layout(
    title='Log of Absolute Deviations for Different Distributions (Monte Carlo Simulation)',
    xaxis_title='Number of Samples',
    yaxis_title='Log10(Absolute Deviation from True Mean)',
    legend_title='Distributions',
    template='plotly_dark',
    hovermode='x unified'
)

# Show the plot
fig.show()

/tmp/ipykernel_1694/566385728.py:58: RuntimeWarning:

divide by zero encountered in log10

